# SQL——Structured Query Language


结构化查询语言SQL（STRUCTURED QUERY LANGUAGE）是最重要的关系数据库操作语言，并且它的影响已经超出数据库领域，得到其他领域的重视和采用，如人工智能领域的数据检索，第四代软件开发工具中嵌入SQL的语言等。


## Basic query in MySQL
###1. CREATE TABLE
create table table_name

(column1 data type(NULL||NOT NULL));

desc: create a table
 
###2. SHOW table
show CREATE TABLE table_name;

desc: show the table in your database
###3. INSERT
insert into table_name

values('value1','value2',......);

desc: insert new rows of data into the database

###4. SELECT 
select * from table table_name.

desc: return data from the table

###5. SELECT
select column1,column2,... 

from table1,table2...

[where conditions]

[group by column1,column2...]

[having conditions]

[order by column1,column2...]

desc: return data from one or more database tables

###6. UPDATE
update table_name set

column1='value1,column2='value2'...

[where conditions]
desc: update existing data in the table

### 7. DELETE
delete from table_name

[where conditions]

desc: delete rows of data from a table


##Innodb表

表空间：表空间可看做是InnoDB存储引擎逻辑结构的最高层。

段：表空间由各个段组成，常见的段有数据段、索引段、回滚段等。

区：由64个连续的页组成，每个页大小为16kb，即每个区大小为1MB。 

页：每页16kb，且不能更改。常见的页类型有：数据页、Undo页、系统页、事务数据页、插入缓冲位图页、插入缓冲空闲列表页、未压缩的二进制大对象页、压缩的二进制大对象页。

行：InnoDB存储引擎是面向行的(row-oriented)，每页最多允许存放7992行数据。 
行记录格式：常见两种行记录格式Compact和Redundant，mysql5.1版本后，主要是Compact行记录格式。对于Compact，不管是char型还是varchar型，null型都是不占用存储空间的；对于Redudant,varchar的null不占用空间，char的null型是占用存储空间的。


## Innodb常见索引与算法

B+树索引：B+树的数据结构相对较复杂，B代表的是balance最早是从平衡二叉树演化而来，但B+树并不是一个二叉树，由于B+树索引的高扇出性，因此在数据库中，B+树的高度一般都在2~3层，也就对于查找某一键值的行记录，最多只要2到3次IO,现在一般的磁盘每秒至少可以做100次IO，2~3次的IO意味着查询时间只需0.02~0.03秒。

数据库中的B+索引可以分为聚集索引(clustered index)和辅助聚集索引(secondary index),但其内部都是B+树的，即高度平衡的，叶子节点存放数据。

聚集索引：由于聚集索引是按照主键组织的，所以每一张表只能有一个聚集索引，每个数据页都通过双向链表进行连接，叶子节点存放一整行的信息，所以查询优化器更倾向走聚集索引。此外，对于聚集索引的存储是逻辑上连续的。所以，聚集索引对于主键的排序查找和范围查找速度非常快。

辅助索引：也叫非聚集索引，叶子节点不存全部数据，主要存键值及一个boomark(其实就是聚集索引的键)告诉InnoDB哪里可以找到与索引相对应的行数据，如一个高度为3的辅助索引和一个高度为3的聚集索引，若根据辅助索引来查询行记录，一共需要6次IO。另外辅助索引可以有多个。

索引的使用原则：高选择、取出表中的少部分数据(也称为唯一索引)。一般取出的数据量超过表中数据的20%，优化器不会使用索引，而进行全表扫描。如对于性别等字段是没有意义的。

联合索引： 也称复合索引，是在多列（>=2）上建立的索引。Innodb中的复合索引也是b+ tree结构。索引的数据包含多列(col1, col2, col3…)，在索引中依次按照col1, col2, col3排序。如(1, 2), (1, 3),(2,0)…使用复合索引要充分利用最左前缀原则，顾名思义，就是最左优先。如创建索引ind_col1_col2(col1, col2)，那么在查询where col1 = xxx and col2 = xx或者where col1 = xxx都可以走ind_col1_col2索引，但where col2=****是走不到索引的。在创建多列索引时，要根据业务需求，where子句中使用最频繁且过滤效果好的的一列放在最左边。
哈希索引：哈希算法也是比较常见的算法，mysql innoDB中使用了比较常见的链地址法进行去重。此外上面已经提及，innoDB中的hash是自适应的，什么时候使用hash是系统决定的，无法进行人工设置。


## Innodb中的锁

InnoDB存储引擎锁的实现和Oracle非常类似，提供一致性的非锁定读、行级锁支持、行级锁没有相关的开销，可以同时得到并发性和一致性。

InnoDB存储引擎实现了如下两种标准的行级锁：

共享锁(S Lock)：允许事务读一行数据； 

排他锁(X Lock)：允许事务删除或者更新一行数据。

当一个事务已经获得了行r的共享锁，那么另外的事务可以立即获得行r的共享锁，因为读取没有改变行r的数据，我们称这种情况为锁兼容。但如果有事务想获得行r的排他锁，则它必须等待事务释放行r上的共享锁————这种情况称为锁不兼容。


## Innodb中的事务

事务的四个特性：原子性、一致性、隔离性、持久性

隔离性通过锁实现，原子性、一致性、持久性通过数据库的redo和undo来完成。

重做日志记录了事务的行为，通过redo实现，保证了事务的完整性，但事务有时还需要撤销，这时就需要产生undo。undo和redo正好相反，对于数据库进行修改时，数据库不但会产生redo，而且还会产生一定的undo，即使执行的事务或语句由于某种原因失败了，或者如果用一条rollback语句请求回滚，就可以用这些undo信息将数据回滚到修改之前的样子。与redo不同的是,redo存放在重做日志文件中，undo存放在数据库内部的一个特殊段(segment)中，这称为undo段(undo segment)，undo段位于共享表空间内。还有一点重要的是，undo记录的是与事务操作相反的逻辑操作，如insert undo 记录一个delete，所以undo只是逻辑地将数据库恢复成事务开始前的样子。如:insert 10万行的数据，可能导致表空间增大，回滚后，表空间不会减小回去。
